In [1]:
using Interact
using CoordinateTransformations
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using ProfileView
using DrakeVisualizer
import Flash
using Ipopt

In [22]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

Process(`/Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/../deps/usr/bin/drake-visualizer`, ProcessRunning)

In [3]:
reload("Flash")

In [21]:
vis = Visualizer()
delete!(vis)

Set{Array{Symbol,1}}()

In [23]:
model = Flash.Models.beanbag()
state = Flash.ManipulatorState(model)
setgeometry!(vis[:robot], model)
settransform!(vis[:robot], state)
Flash.draw_skin!(vis[:skin], state)

In [24]:
geometry = model.surfaces[1]
@manipulate for x = linspace(-1.01, 0.99, 51)
    state.deformations[geometry][1][:] = [x, 0, 0]
    Flash.draw_skin!(vis[:skin], state)
end;

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(-0.01, nactions=1),"x",-0.01,"-0.01",Interact.OptionDict(DataStructures.OrderedDict("-1.01"=>-1.01,"-0.97"=>-0.97,"-0.93"=>-0.93,"-0.89"=>-0.89,"-0.85"=>-0.85,"-0.81"=>-0.81,"-0.77"=>-0.77,"-0.73"=>-0.73,"-0.69"=>-0.69,"-0.65"=>-0.65…),Dict(-0.45=>"-0.45",0.35=>"0.35",-0.29=>"-0.29",-0.85=>"-0.85",-0.17=>"-0.17",0.67=>"0.67",0.71=>"0.71",0.83=>"0.83",-0.97=>"-0.97",-0.61=>"-0.61"…)),Any[],Any[],true,"horizontal")

In [25]:
# Construct the sensor and draw its view rays for debugging
sensor = Flash.DepthSensors.Kinect(80,80);
camera_origin = [0., 0, 8]
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi, 1, 0, 0)))
Flash.DepthSensors.draw_rays(vis[:sensor], sensor, camera_tform)

Set{Array{Symbol,1}}()

In [31]:
true_state = Flash.ManipulatorState(model)
true_state.mechanism_state.q[5:7] = 2 * rand(3)
true_state.deformation_data[:] = 0.5 .* (rand(Float64, length(true_state.deformation_data)) .- 0.5)
settransform!(vis[:robot], true_state)
Flash.draw_skin!(vis[:skin], true_state)

In [32]:
# Now we can raycast that true state to get a set of sensed points:
sensed_points = Flash.DepthSensors.raycast(true_state, sensor, camera_tform)
setgeometry!(vis[:sensed_points], PointCloud(sensed_points))

Visualizer with path prefix Symbol[:sensed_points] using LCM LCMCore.LCM(Ptr{Void} @0x00007fd4afb59db0,"",RawFD(60),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x00007fd4adbd8970)])

In [33]:
function test_gradient_descent(model, sensed_points, true_state, loops)
    x_true = copy(Flash.flatten(true_state))
    trials = Vector{DataFrame}()
    solver = NaiveSolver(length(x_true); rate=1.0, max_step=0.1, iteration_limit=2000)
#     solver = IpoptSolver()
    
    for i = 1:loops
        xs = Vector{Float64}[]
        costs = Float64[]
        
        function callback{T}(x::AbstractVector{T}, c)
            x_value = Flash.value.(x)
            state = Flash.ManipulatorState(model)
            Flash.GradientDescent.unflatten!(state, x_value)
            settransform!(vis[:robot], state)
            push!(xs, x_value)
            push!(costs, Flash.value(c))
        end
        
        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        Flash.Tracking.estimate_state(model, sensed_points, x_estimated; callback=callback, solver=solver)
        push!(trials, DataFrame(x=xs, cost=costs))
        
        state = Flash.ManipulatorState(model)
        Flash.GradientDescent.unflatten!(state, xs[end])
        settransform!(vis[:robot], state)
        Flash.draw_skin!(vis[:skin], state)
    end
    trials
end


test_gradient_descent (generic function with 1 method)

In [35]:
test_gradient_descent(model, sensed_points, true_state, 1);